In [3]:
from stellargraph import StellarDiGraph

In [2]:
import numpy as np
import pandas as pd

In [152]:
def create_df():
    data = {'Source':[], 'Sink':[]}
    with open("train.txt", "r") as f:
        for raw_line in f:
            line = raw_line.split("\t")
            data['Source'] += [line[0]]*(len(line)-1)
            data['Sink'] += line[1:]
    pd_data = pd.DataFrame(data=data)
#     pd_data[['Source', 'Sink']] = pd_data[['Source', 'Sink']].apply(pd.to_numeric)
    pd_data = pd_data.drop_duplicates(keep=False)
    return pd_data

In [153]:
graph_df = create_df()

In [154]:
graph_df.shape

(23888876, 2)

In [201]:
graph_df.head()

,Source,Sink
0,540762,1912140
1,540762,1537559
2,540762,3091331
3,540762,2757277
4,540762,3237295


In [202]:
reduced_df = graph_df[(graph_df['Source'].isin(sub_nodes)) | (graph_df['Sink'].isin(sub_nodes))]

In [203]:
reduced_df.shape

(5035729, 2)

In [158]:
graph = StellarDiGraph(edges=graph_df, 
                       source_column="Source", 
                       target_column="Sink",
                      )

In [159]:
print(graph.info())

StellarDiGraph: Directed multigraph
 Nodes: 4854068, Edges: 23888876

 Node types:
  default: [4854068]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [23888876]
        Weights: all 1 (default)
        Features: none


In [ ]:
reduced_df = graph_df.sample(100000)

In [129]:
further_reduced_df.shape

(100000, 2)

In [130]:
graph = StellarDiGraph(edges=further_reduced_df, 
                       source_column="Source", 
                       target_column="Sink",
                      )

In [131]:
print(graph.info())

StellarDiGraph: Directed multigraph
 Nodes: 8988, Edges: 100000

 Node types:
  default: [8988]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [100000]
        Weights: all 1 (default)
        Features: none


In [ ]:
graph.out_degree()

# Node2Vec

In [132]:
import matplotlib.pyplot as plt
from math import isclose
from sklearn.decomposition import PCA
import os
import networkx as nx
import numpy as np
import pandas as pd
from stellargraph import StellarGraph, datasets
from stellargraph.data import EdgeSplitter
from collections import Counter
import multiprocessing
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split

%matplotlib inline

In [133]:
# Define an edge splitter on the original graph:
edge_splitter_test = EdgeSplitter(graph)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from graph, and obtain the
# reduced graph graph_test with the sampled links removed:
graph_test, examples_test, labels_test = edge_splitter_test.train_test_split(
    p=0.1, method="global"
)

print(graph_test.info())

** Sampled 10000 positive and 10000 negative edges. **
StellarDiGraph: Directed multigraph
 Nodes: 8988, Edges: 90000

 Node types:
  default: [8988]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [90000]
        Weights: all 1 (default)
        Features: none


In [134]:
# Do the same process to compute a training subset from within the test graph
edge_splitter_train = EdgeSplitter(graph_test, graph)
graph_train, examples, labels = edge_splitter_train.train_test_split(
    p=0.1, method="global"
)

(
    examples_train,
    examples_model_selection,
    labels_train,
    labels_model_selection,
) = train_test_split(examples, labels, train_size=0.75, test_size=0.25)

print(graph_train.info())

** Sampled 9000 positive and 9000 negative edges. **
StellarDiGraph: Directed multigraph
 Nodes: 8988, Edges: 81000

 Node types:
  default: [8988]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [81000]
        Weights: all 1 (default)
        Features: none


In [135]:
pd.DataFrame(
    [
        (
            "Training Set",
            len(examples_train),
            "Train Graph",
            "Test Graph",
            "Train the Link Classifier",
        ),
        (
            "Model Selection",
            len(examples_model_selection),
            "Train Graph",
            "Test Graph",
            "Select the best Link Classifier model",
        ),
        (
            "Test set",
            len(examples_test),
            "Test Graph",
            "Full Graph",
            "Evaluate the best Link Classifier",
        ),
    ],
    columns=("Split", "Number of Examples", "Hidden from", "Picked from", "Use"),
).set_index("Split")

,Number of Examples,Hidden from,Picked from,Use
Split,,,,
Training Set,13500,Train Graph,Test Graph,Train the Link Classifier
Model Selection,4500,Train Graph,Test Graph,Select the best Link Classifier model
Test set,20000,Test Graph,Full Graph,Evaluate the best Link Classifier


In [136]:
p = 1.0
q = 1.0
dimensions = 128
num_walks = 10
walk_length = 80
window_size = 10
num_iter = 1
workers = 6

In [137]:
from stellargraph.data import BiasedRandomWalk
from gensim.models import Word2Vec


def node2vec_embedding(graph, name):
    rw = BiasedRandomWalk(graph)
    walks = rw.run(graph.nodes(), n=num_walks, length=walk_length, p=p, q=q)
    print(f"Number of random walks for '{name}': {len(walks)}")

    model = Word2Vec(
        walks,
        size=dimensions,
        window=window_size,
        min_count=0,
        sg=1,
        workers=workers,
        iter=num_iter,
    )

    def get_embedding(u):
        return model.wv[u]

    return get_embedding

In [138]:
embedding_train = node2vec_embedding(graph_train, "Train Graph")

Number of random walks for 'Train Graph': 89880


In [139]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler


# 1. link embeddings
def link_examples_to_features(link_examples, transform_node, binary_operator):
    return [
        binary_operator(transform_node(src), transform_node(dst))
        for src, dst in link_examples
    ]


# 2. training classifier
def train_link_prediction_model(
    link_examples, link_labels, get_embedding, binary_operator
):
    clf = link_prediction_classifier()
    link_features = link_examples_to_features(
        link_examples, get_embedding, binary_operator
    )
    clf.fit(link_features, link_labels)
    return clf


def link_prediction_classifier(max_iter=2000):
    lr_clf = LogisticRegressionCV(Cs=10, cv=10, scoring="roc_auc", max_iter=max_iter)
    return Pipeline(steps=[("sc", StandardScaler()), ("clf", lr_clf)])


# 3. and 4. evaluate classifier
def evaluate_link_prediction_model(
    clf, link_examples_test, link_labels_test, get_embedding, binary_operator
):
    link_features_test = link_examples_to_features(
        link_examples_test, get_embedding, binary_operator
    )
    score = evaluate_roc_auc(clf, link_features_test, link_labels_test)
    return score


def evaluate_roc_auc(clf, link_features, link_labels):
    predicted = clf.predict_proba(link_features)

    # check which class corresponds to positive links
    positive_column = list(clf.classes_).index(1)
    return roc_auc_score(link_labels, predicted[:, positive_column])

In [140]:
def operator_hadamard(u, v):
    return u * v


def operator_l1(u, v):
    return np.abs(u - v)


def operator_l2(u, v):
    return (u - v) ** 2


def operator_avg(u, v):
    return (u + v) / 2.0


def run_link_prediction(binary_operator):
    clf = train_link_prediction_model(
        examples_train, labels_train, embedding_train, binary_operator
    )
    score = evaluate_link_prediction_model(
        clf,
        examples_model_selection,
        labels_model_selection,
        embedding_train,
        binary_operator,
    )

    return {
        "classifier": clf,
        "binary_operator": binary_operator,
        "score": score,
    }


binary_operators = [operator_hadamard, operator_l1, operator_l2, operator_avg]

In [141]:
results = [run_link_prediction(op) for op in binary_operators]
best_result = max(results, key=lambda result: result["score"])

print(f"Best result from '{best_result['binary_operator'].__name__}'")

pd.DataFrame(
    [(result["binary_operator"].__name__, result["score"]) for result in results],
    columns=("name", "ROC AUC score"),
).set_index("name")

Best result from 'operator_hadamard'


,ROC AUC score
name,
operator_hadamard,0.820715
operator_l1,0.771053
operator_l2,0.773341
operator_avg,0.721774


In [142]:
embedding_test = node2vec_embedding(graph_test, "Test Graph")

Number of random walks for 'Test Graph': 89880


In [143]:
test_score = evaluate_link_prediction_model(
    best_result["classifier"],
    examples_test,
    labels_test,
    embedding_test,
    best_result["binary_operator"],
)
print(
    f"ROC AUC score on test set using '{best_result['binary_operator'].__name__}': {test_score}"
)

ROC AUC score on test set using 'operator_hadamard': 0.76363176


# Prediction

In [173]:
def read_sub():
    with open('test-public.txt', 'r') as f:
        # skip the header
        f.readline()
        data = {'Source':[], 'Sink':[]}
        for raw_line in f:
            line = raw_line.strip().split("\t")
            data['Source'].append(line[1])
            data['Sink'].append(line[2])
        return pd.DataFrame(data=data)

In [174]:
sub_data = read_sub()

In [175]:
sub_data.head()

,Source,Sink
0,3563811,3600160
1,2052043,1401960
2,4517994,1690636
3,1660006,4349447
4,581111,1882617


In [199]:
sub_nodes = set(sub_data['Source'].value_counts().keys()).union(sub_data['Sink'].value_counts().keys())

In [176]:
sub_graph = StellarDiGraph(edges=sub_data, 
                       source_column="Source", 
                       target_column="Sink",
                      )

print(sub_graph.info())

StellarDiGraph: Directed multigraph
 Nodes: 3948, Edges: 2000

 Node types:
  default: [3948]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [2000]
        Weights: all 1 (default)
        Features: none
